# Quora Question Pairs

# Вступление


In [1]:
# Imports
#from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
#import seaborn
import pandas as pd
from sklearn.preprocessing import normalize
from gensim import utils
#from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec as Doc2Vec
from gensim.models import KeyedVectors as KeyedVectors
#import seaborn
from random import shuffle
%matplotlib inline


In [2]:
data_dir = '/home/uasa/Desktop/data/'
google_vec_filepath = data_dir + 'GoogleNews-vectors-negative300.bin'
lexvec_filepath = data_dir + 'lexvec.enwiki+newscrawl.300d.W+C.pos.vectors'
quora_train_filepath = data_dir + 'train.csv'

In [3]:
# Load test and train data
#test = pd.read_csv("./test.csv")
train = pd.read_csv(quora_train_filepath)

In [4]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
train.dropna(inplace = True)
train.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
import string

def remove_punctuation(s):
    s = ''.join([i.lower() for i in s if i not in frozenset(string.punctuation)])
    return s

def clean_punct(series):
    list1 = []
    list2 = series.tolist()
    for item in list2:
        list1.append(remove_punctuation(item))
    return list1



In [7]:
train['question2'] = train['question2'].apply(remove_punctuation)
train['question1'] = train['question1'].apply(remove_punctuation)
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


In [8]:
train['question1'] = train['question1'].str.split()
train['question2'] = train['question2'].str.split()
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...",0
1,1,3,4,"[what, is, the, story, of, kohinoor, kohinoor,...","[what, would, happen, if, the, indian, governm...",0
2,2,5,6,"[how, can, i, increase, the, speed, of, my, in...","[how, can, internet, speed, be, increased, by,...",0
3,3,7,8,"[why, am, i, mentally, very, lonely, how, can,...","[find, the, remainder, when, math2324math, is,...",0
4,4,9,10,"[which, one, dissolve, in, water, quikly, suga...","[which, fish, would, survive, in, salt, water]",0


In [9]:
def get_unique_parts(df):
    l1 = []
    l2 = []
    for row in df.itertuples():
        l1.append(list(set(row[4]).difference(row[5])))
        l2.append(list(set(row[5]).difference(row[4])))
    new_frame = pd.DataFrame(
    {'first_unique': l1,
     'second_unique': l2,
    })
    return new_frame

In [10]:
delta = get_unique_parts(train)

In [11]:
delta.head()

,first_unique,second_unique
0,[india],[]
1,"[story, of, is]","[would, government, back, indian, stole, happe..."
2,"[a, i, of, connection, increase, while, using,...","[be, increased, through, dns, hacking, by]"
3,"[lonely, i, am, it, how, solve, can, very, why...","[is, 2423, when, find, the, math2324math, rema..."
4,"[and, di, one, oxide, quikly, carbon, sugar, d...","[would, survive, fish]"


# TF-IDF

In [12]:
tfidf_data = pd.DataFrame()
tfidf_data['text'] = train['question1'] + train['question2']
tfidf_data['text'] = tfidf_data['text'].apply(lambda x: ' '.join(x))
tfidf_data.head()

,text
0,what is the step by step guide to invest in sh...
1,what is the story of kohinoor kohinoor diamond...
2,how can i increase the speed of my internet co...
3,why am i mentally very lonely how can i solve ...
4,which one dissolve in water quikly sugar salt ...


In [13]:
#stemming - trying to drop all word endings
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
stemmed = tfidf_data.text.map(lambda x: ' '.join([stemmer.stem(y) for y in x.decode('utf-8').split(' ')]))

In [14]:
stemmed.head()

0    what is the step by step guid to invest in sha...
1    what is the stori of kohinoor kohinoor diamond...
2    how can i increas the speed of my internet con...
3    whi am i mental veri lone how can i solv it fi...
4    which one dissolv in water quik sugar salt met...
Name: text, dtype: object

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [16]:
#cvec.fit(tfidf_data.text)
cvec = CountVectorizer(stop_words='english',min_df=2, max_df=0.1)
cvec.fit(stemmed)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.1, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [17]:
len(cvec.vocabulary_)

40439

In [18]:
cvec_counts = cvec.transform(stemmed)
print ('sparse matrix shape:', cvec_counts.shape)
print ('nonzero count:', cvec_counts.nnz)
print ('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

('sparse matrix shape:', (404288, 40439))
('nonzero count:', 2906708)
sparsity: 0.02%


In [19]:
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(10)

,occurrences,term
18360,29387,india
21336,28260,like
37835,27041,use
27502,26496,peopl
15751,25066,good
38849,24979,way
11305,24091,differ
22184,23293,make
29625,18175,quora
21016,17945,learn


In [20]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<404288x40439 sparse matrix of type '<type 'numpy.float64'>'
	with 2906708 stored elements in Compressed Sparse Row format>

In [21]:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=True).head(10)

,term,weight
28398,porblem,7.077604e-07
1882,4x17,8.101529e-07
1040,20cm,8.110702e-07
17258,hollowpoint,8.688289e-07
21919,m265,8.831492e-07
35343,tambola,8.833273e-07
31560,saluki,8.881040e-07
596,1720,9.041358e-07
15074,gamecock,9.058218e-07
15390,gerri,9.130964e-07


In [23]:
weights_df.to_csv(data_dir+'tfidf.csv', encoding='utf8')

In [22]:
weights_df[weights_df['weight'] > 0.0001].sort_values(by='weight', ascending=True).head(10)

,term,weight
22189,maker,0.000100
20381,knight,0.000100
6219,bitter,0.000100
18157,immun,0.000100
18952,ipo,0.000100
33869,spent,0.000100
28534,potus,0.000101
19641,journalist,0.000101
27884,pickup,0.000101
26304,oneplus,0.000101


In [23]:
weights_df[weights_df['term']=='kushi'].empty

True

In [24]:
stemmer.stem('immune')

u'immun'

In [25]:
len(delta)

404288

In [26]:
test_list = ['one','two','three','immune','indians']
for item in test_list:
    item = stemmer.stem(item)
test_list

['one', 'two', 'three', 'immune', 'indians']

In [27]:
def trim_unique(df):
    ll1 = []
    ll2 = []
    for row in df.itertuples():
        l1 = []
        l2 = []
        for item in row[1]:
            l1.append(stemmer.stem(item.decode('utf-8')))
        for item in row[2]:
            l2.append(stemmer.stem(item.decode('utf-8')))
        ll1.append(l1)
        ll2.append(l2)
    new_frame = pd.DataFrame(
    {'first_trimmed': ll1,
     'second_trimmed': ll2,
    })
    return new_frame

In [28]:
delta = delta.join(trim_unique(delta))

In [29]:
delta.head(10)

,first_unique,second_unique,first_trimmed,second_trimmed
0,[india],[],[india],[]
1,"[story, of, is]","[would, government, back, indian, stole, happe...","[stori, of, is]","[would, govern, back, indian, stole, happen, if]"
2,"[a, i, of, connection, increase, while, using,...","[be, increased, through, dns, hacking, by]","[a, i, of, connect, increas, while, use, the, ...","[be, increas, through, dns, hack, by]"
3,"[lonely, i, am, it, how, solve, can, very, why...","[is, 2423, when, find, the, math2324math, rema...","[lone, i, am, it, how, solv, can, veri, whi, m...","[is, 2423, when, find, the, math2324math, rema..."
4,"[and, di, one, oxide, quikly, carbon, sugar, d...","[would, survive, fish]","[and, di, one, oxid, quik, carbon, sugar, diss...","[would, surviv, fish]"
5,"[that, i, cap, am, risingwhat, astrology]","[what, ascendant, this, im, triple, in]","[that, i, cap, am, risingwhat, astrolog]","[what, ascend, this, im, tripl, in]"
6,"[i, buy, tiago, should]","[and, what, from, childern, far, phone, video,...","[i, buy, tiago, should]","[and, what, from, childern, far, phone, video,..."
7,"[how, good, can]","[do, what, great, should, to]","[how, good, can]","[do, what, great, should, to]"
8,"[し, シ]",[and],"[し, シ]",[and]
9,"[charter, company, can, motorolla, my]","[do, for, internet, free, how]","[charter, compani, can, motorolla, my]","[do, for, internet, free, how]"


In [30]:
train.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...",0
1,1,3,4,"[what, is, the, story, of, kohinoor, kohinoor,...","[what, would, happen, if, the, indian, governm...",0
2,2,5,6,"[how, can, i, increase, the, speed, of, my, in...","[how, can, internet, speed, be, increased, by,...",0
3,3,7,8,"[why, am, i, mentally, very, lonely, how, can,...","[find, the, remainder, when, math2324math, is,...",0
4,4,9,10,"[which, one, dissolve, in, water, quikly, suga...","[which, fish, would, survive, in, salt, water]",0
5,5,11,12,"[astrology, i, am, a, capricorn, sun, cap, moo...","[im, a, triple, capricorn, sun, moon, and, asc...",1
6,6,13,14,"[should, i, buy, tiago]","[what, keeps, childern, active, and, far, from...",0
7,7,15,16,"[how, can, i, be, a, good, geologist]","[what, should, i, do, to, be, a, great, geolog...",1
8,8,17,18,"[when, do, you, use, シ, instead, of, し]","[when, do, you, use, instead, of, and]",0
9,9,19,20,"[motorola, company, can, i, hack, my, charter,...","[how, do, i, hack, motorola, dcx3400, for, fre...",0


In [31]:
train.loc[5, 'question1']

['astrology',
 'i',
 'am',
 'a',
 'capricorn',
 'sun',
 'cap',
 'moon',
 'and',
 'cap',
 'risingwhat',
 'does',
 'that',
 'say',
 'about',
 'me']

In [32]:
train.loc[5, 'question2']

['im',
 'a',
 'triple',
 'capricorn',
 'sun',
 'moon',
 'and',
 'ascendant',
 'in',
 'capricorn',
 'what',
 'does',
 'this',
 'say',
 'about',
 'me']

### all together with keyed vector load is ~10 minutes

# Word embeddings

In [33]:
word_vectors = KeyedVectors.load_word2vec_format(lexvec_filepath)

In [34]:
word_vectors.similarity('indian','india')

0.69059315947651323

In [35]:
word_vectors.most_similar('india')

[(u'indian', 0.6905931234359741),
 (u'delhi', 0.6824976205825806),
 (u'subcontinent', 0.6603225469589233),
 (u'pradesh', 0.6530251502990723),
 (u'gujarat', 0.6477441787719727),
 (u'maharashtra', 0.6461658477783203),
 (u'mumbai', 0.6329033374786377),
 (u'uttar', 0.62990403175354),
 (u'bengal', 0.6288950443267822),
 (u'haryana', 0.6189148426055908)]

In [125]:
test = weights_df.set_index('term')
test.loc['suzann', 'weight']
if 'suzann' in test.index:
    print(True)
else:
    print(False)

True


In [126]:
weights_df = weights_df.set_index('term')

In [127]:
def check_value(item):
    if item in weights_df.index:
        return weights_df.loc[item,'weight']
    return 0

def count_tfidf(df):
    ll1 = []
    ll2 = []
    for row in df.itertuples():
        l1 = []
        l2 = []
        for item in row[3]:
            l1.append(check_value(item))
        for item in row[4]:
            l2.append(check_value(item))
        ll1.append(l1)
        ll2.append(l2)
    new_frame = pd.DataFrame(
    {'first_tfidf': ll1,
     'second_tfidf': ll2,
    })
    return new_frame

In [130]:
test = delta[:10]

test

,first_unique,second_unique,first_trimmed,second_trimmed
0,[india],[],[india],[]
1,"[story, of, is]","[would, government, back, indian, stole, happe...","[stori, of, is]","[would, govern, back, indian, stole, happen, if]"
2,"[a, i, of, connection, increase, while, using,...","[be, increased, through, dns, hacking, by]","[a, i, of, connect, increas, while, use, the, ...","[be, increas, through, dns, hack, by]"
3,"[lonely, i, am, it, how, solve, can, very, why...","[is, 2423, when, find, the, math2324math, rema...","[lone, i, am, it, how, solv, can, veri, whi, m...","[is, 2423, when, find, the, math2324math, rema..."
4,"[and, di, one, oxide, quikly, carbon, sugar, d...","[would, survive, fish]","[and, di, one, oxid, quik, carbon, sugar, diss...","[would, surviv, fish]"
5,"[that, i, cap, am, risingwhat, astrology]","[what, ascendant, this, im, triple, in]",[astrolog],[im]
6,"[i, buy, tiago, should]","[and, what, from, childern, far, phone, video,...","[i, buy, tiago, should]","[and, what, from, childern, far, phone, video,..."
7,"[how, good, can]","[do, what, great, should, to]","[how, good, can]","[do, what, great, should, to]"
8,"[し, シ]",[and],"[し, シ]",[and]
9,"[charter, company, can, motorolla, my]","[do, for, internet, free, how]","[charter, compani, can, motorolla, my]","[do, for, internet, free, how]"


In [131]:
count_tfidf(test)

,first_tfidf,second_tfidf
0,[0.0110155138276],[]
1,"[0.00156110791557, 0, 0]","[0, 0.00204569964604, 0, 0.00530077916237, 2.3..."
2,"[0, 0, 0, 0.000941165675423, 0.00276639763198,...","[0, 0.00276639763198, 0, 2.95835727559e-05, 0...."
3,"[0.000253797172722, 0, 0, 0, 0, 0.001145867409...","[0, 0, 0, 0, 0, 0, 0.00010966302079, 0, 0.0002..."
4,"[0, 2.46501316168e-05, 0, 0.000163478776817, 3...","[0, 0.000572156825079, 0.000345479147768]"
5,[0.00044574957611],[0.00267213058054]
6,"[0, 0.00374998234923, 4.17942108044e-06, 0]","[0, 0, 0, 0, 0.000533748909895, 0.003863214217..."
7,"[0, 0.00931161461492, 0]","[0, 0, 0.00148150275744, 0, 0]"
8,"[0, 0]",[0]
9,"[0.000182191749139, 0.00383245530379, 0, 0, 0]","[0, 0, 0.00156771690361, 0.00284080040679, 0]"


In [90]:
def check_vocab(x, y):
    if x in word_vectors.vocab:
        if y in word_vectors.vocab:
            return word_vectors.similarity(x, y)
    return 0

def count_w2v(df):
    ll1 = []
    for row in df.itertuples():
        l1 = []
        for item1 in row[1]:
            for item2 in row[2]:
                l1.append(check_vocab(item1, item2))
        ll1.append(l1)
    new_frame = pd.DataFrame(
    {'word2vec': ll1,
    })
    return new_frame

In [91]:
count_w2v(test)

,word2vec
0,[]
1,"[0.121086688011, -0.00260932710238, 0.19284702..."
2,"[0.549211967375, 0.187426972881, 0.36903241766..."
3,"[0.176601278828, 0, 0.184140951803, 0.28165582..."
4,"[0.441143084605, 0.246326873193, 0.22918478869..."
5,"[0.712231425218, 0.0908104200734, 0.7258445797..."
6,"[0.350023546714, 0.642018438972, 0.16033757603..."
7,"[0.598143059377, 0.790946151682, 0.27648299867..."
8,"[0, 0]"
9,"[0.0325456114579, 0.188586949925, 0.1704522258..."


In [134]:
train = train.join([count_tfidf(delta),count_w2v(delta)])
#tfidf - 105 msec per 100 elements, 10.5 seconds per 10000 elements. 9 minutes per 500000. Meh
#w2v - 75.8 msec per 100 elements, 7.41 seconds per 10000 elements, 6 minutes per 500000. Acceptable

In [135]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,first_tfidf,second_tfidf,word2vec
0,0,1,2,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...",0,[0.0110155138276],[],[]
1,1,3,4,"[what, is, the, story, of, kohinoor, kohinoor,...","[what, would, happen, if, the, indian, governm...",0,"[0.00156110791557, 0, 0]","[0, 0.00204569964604, 0, 0.00530077916237, 2.3...","[0.121086688011, -0.00260932710238, 0.19284702..."
2,2,5,6,"[how, can, i, increase, the, speed, of, my, in...","[how, can, internet, speed, be, increased, by,...",0,"[0, 0, 0, 0.000941165675423, 0.00276639763198,...","[0, 0.00276639763198, 0, 2.95835727559e-05, 0....","[0.549211967375, 0.187426972881, 0.36903241766..."
3,3,7,8,"[why, am, i, mentally, very, lonely, how, can,...","[find, the, remainder, when, math2324math, is,...",0,"[0.000253797172722, 0, 0, 0, 0, 0.001145867409...","[0, 0, 0, 0, 0, 0, 0.00010966302079, 0, 0.0002...","[0.176601278828, 0, 0.184140951803, 0.28165582..."
4,4,9,10,"[which, one, dissolve, in, water, quikly, suga...","[which, fish, would, survive, in, salt, water]",0,"[0, 2.46501316168e-05, 0, 0.000163478776817, 3...","[0, 0.000572156825079, 0.000345479147768]","[0.441143084605, 0.246326873193, 0.22918478869..."


In [136]:
train.to_csv(data_dir + 'generated_features.csv')